In [12]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import random

# Set seed for reproducibility
np.random.seed(0)

# Parameters
num_days = 360
initial_capital = 10000
max_shares_per_trade = 400  # Maximum number of shares per transaction
price_range = (0, 30)  # Assumed stock price range
n = 15  # Number of days for moving average calculation

# Generate stock prices
def generate_stock_prices(num_days):
    prices = []
    for i in range(1, num_days + 1):
        zi = 0.5 * random.uniform(-1, 1) * (i % 3) + 2.0 * random.uniform(-1, 1) * (i % 11)
        price = 10 + 2.8 * np.sin(2 * np.pi * i / 19) + 0.9 * np.cos(2 * np.pi * i / 13) + zi + 0.016 * i
        prices.append(price)
    return prices

# Calculate moving averages
def calculate_moving_averages(prices, n):
    return np.convolve(prices, np.ones(n, dtype=int), 'valid') / n

# Define the trend indicator
def calculate_trend(prices):
    trends = np.diff(prices)
    return trends

# Generate the prices and indicators
prices = generate_stock_prices(num_days)
moving_averages = calculate_moving_averages(prices, n)
trends = calculate_trend(prices)
trends = np.insert(trends, 0, 0)  # To match the prices' array length

# Fuzzy variables
price = ctrl.Antecedent(np.linspace(*price_range, num=1000), 'price')
ma = ctrl.Antecedent(np.linspace(*price_range, num=1000), 'ma')
trend = ctrl.Antecedent(np.linspace(-2, 2, num=1000), 'trend')
action = ctrl.Consequent(np.arange(-2, 3, 1), 'action')

# Membership functions for Price, MA, Trend, and Action
price.automf(5, names=['VL', 'LO', 'MD', 'HI', 'VH'])  # Very Low, Low, Medium, High, Very High
ma.automf(5, names=['VL', 'LO', 'MD', 'HI', 'VH'])
trend.automf(3, names=['negative', 'steady', 'positive'])

action.automf(5, names=['SM', 'SF', 'DT', 'BF', 'BM'])  # Strong Sell, Sell, Do Nothing, Buy, Strong Buy

# Rule system - making sure to cover all possible situations to avoid sparse rulebase.
rules = [
    ctrl.Rule((price['VL'] | ma['VL']) & trend['negative'], action['SM']),
    ctrl.Rule((price['VL'] | ma['VL']) & trend['steady'], action['SF']),
    ctrl.Rule((price['VL'] | ma['VL']) & trend['positive'], action['DT']),

    ctrl.Rule((price['LO'] | ma['LO']) & trend['negative'], action['SF']),
    ctrl.Rule((price['LO'] | ma['LO']) & trend['steady'], action['DT']),
    ctrl.Rule((price['LO'] | ma['LO']) & trend['positive'], action['BF']),

    ctrl.Rule((price['MD'] | ma['MD']) & trend['negative'], action['DT']),
    ctrl.Rule((price['MD'] | ma['MD']) & trend['steady'], action['DT']),
    ctrl.Rule((price['MD'] | ma['MD']) & trend['positive'], action['BF']),

    ctrl.Rule((price['HI'] | ma['HI']) & trend['negative'], action['SF']),
    ctrl.Rule((price['HI'] | ma['HI']) & trend['steady'], action['BF']),
    ctrl.Rule((price['HI'] | ma['HI']) & trend['positive'], action['BM']),

    ctrl.Rule((price['VH'] | ma['VH']) & trend['negative'], action['SM']),
    ctrl.Rule((price['VH'] | ma['VH']) & trend['steady'], action['BF']),
    ctrl.Rule((price['VH'] | ma['VH']) & trend['positive'], action['BM']),
]


# Create control system and simulation
trading_ctrl = ctrl.ControlSystem(rules)
trading_sim = ctrl.ControlSystemSimulation(trading_ctrl)

# Trading simulation from day 51
capital = initial_capital
num_shares = 0

for i in range(50, num_days):
    trading_sim.input['price'] = prices[i]
    trading_sim.input['ma'] = moving_averages[i - n] if i >= n else np.mean(prices[:i])
    trading_sim.input['trend'] = trends[i]

    # Compute the decision
    try:
        trading_sim.compute()
    except ValueError as e:
        print(f"Warning: {str(e)} - Using default action.")
        trading_sim.output['action'] = 0

    decision = trading_sim.output['action']
    trade_volume = min(max_shares_per_trade, capital // prices[i])  # Limit the trade volume

    # Execute trading strategy based on 'decision'
    if decision > 1 and capital >= prices[i] * trade_volume:  # Buy conditions
        trade_volume = min(trade_volume, max_shares_per_trade - num_shares)  # Ensure total shares don't exceed max
        capital -= prices[i] * trade_volume
        num_shares += trade_volume
    elif decision < -1 and num_shares >= trade_volume:  # Sell conditions
        trade_volume = min(trade_volume, num_shares)  # Do not sell more than you hold
        capital += prices[i] * trade_volume
        num_shares -= trade_volume

    # End-of-day summary
    print(f"End of Day {i+1} - Capital: {capital}, Num Shares: {num_shares}, Portfolio Value: {capital + num_shares * prices[i]}")

# Final portfolio value
final_portfolio_value = capital + num_shares * prices[-1]
print("Final portfolio value: ${:.2f}".format(final_portfolio_value))


End of Day 51 - Capital: 10000, Num Shares: 0, Portfolio Value: 10000.0
End of Day 52 - Capital: 10000, Num Shares: 0, Portfolio Value: 10000.0
End of Day 53 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 10000.0
End of Day 54 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 55585.09655539887
End of Day 55 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 61691.3969568819
End of Day 56 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 69590.38127234798
End of Day 57 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 82850.94175147876
End of Day 58 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 48918.60611214515
End of Day 59 - Capital: 20000.846113811873, Num Shares: 4470.0, Portfolio Value: 61255.34976164534
End of Day 60 - Capital: 81411.25280822886, Num Shares: 400.0, Portfolio Value: 87446.67361357698
End of Day 61 - Capital: 81411.25280822886, Num Shares: 400.0, Portfo